In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from scipy.stats import norm
from bs4 import BeautifulSoup

In [2]:
def implied_volatility(target_value, call, s, k, t, r):
    limit = 100
    max_error = 1.0e-5
    sigma = 0.5

    for i in range(0, limit):
        price = bs(call, s, k, t, r, sigma)
        vega = vega(call, s, k, t, r, sigma)

        diff = target_value - price

        if (abs(diff) < max_error):
            return sigma
        sigma = sigma + diff/vega

    return sigma

def bs(call, s, k, t, r, v, q=0.0) -> float:
    d1 = (math.log(s / k) + (r + 0.5 * v**2) * t) / (v * math.sqrt(t))
    d2 = d1 - v * math.sqrt(t)
    if call:
        price = s * math.exp(-q * t) * norm.cdf(d1) \
                - k * math.exp(-r * t) * norm.cdf(d2)
    else:
        price = k * math.exp(-r * t) * norm.cdf(-d2) \
                - s * math.exp(-q * t) * norm.cdf(-d1)
    return price

def vega(s, k, t, r, v) -> float:
    d1 = (math.log(s / k) + (r + 0.5 * v**2) * t) / (v * math.sqrt(t))
    return s * math.sqrt(t) * norm.pdf(d1)

In [3]:
frame = pd.read_csv("csv/SI_D_ROPC.txt", sep="|", header=None, skiprows=1,
                    parse_dates=[3], 
                    names=["?", "stock", "stock type", "exercise date", "??", "???",
                           "series", "????", "strike price", "?????", "covered",
                           "uncovered", "total", "owner", "releaser", "??????",
                           "option type"])

frame[frame["stock"]=="PETR"].sort_values(by=["exercise date", "series"])
frame.filter(["stock", "exercise date", "series", "strike price"])
frame["call"] = frame["series"].str.get(4) < "L"

In [4]:
with open("html/Opções de compra | Valor Econômico.html") as f:
    html_string = f.read()
soup = BeautifulSoup(html_string, 'lxml')
table = soup.find_all('table')[0]

In [5]:
table

<table class="valor_tabela"><thead class="tabela-cabecalho-1"><tr class="row row-0 even first table-header"><th class="col col-0 first"><span></span></th><th class="col col-1"><span></span></th><th class="col col-2"><span></span></th><th class="col col-3"><span></span></th><th class="col col-4" colspan="3"><span>(Preços R$)</span></th><th class="col col-5"><span></span></th><th class="col col-6" colspan="1"><span>Oscil.</span></th><th class="col col-7" colspan="2"><span>Ofertas</span></th><th class="col col-8 last" colspan="2"><span>Neg. Realiz.</span></th></tr><tr class="row row-1 odd table-header"><th class="col col-0 first"><span>Cód.</span></th><th class="col col-1"><span>Emp./Ação</span></th><th class="col col-2"><span>Vcto.</span></th><th class="col col-3"><span>Preço</span></th><th class="col col-4"><span>Mín.</span></th><th class="col col-5"><span>Méd.</span></th><th class="col col-6"><span>Max.</span></th><th class="col col-7"><span>Fech.</span></th><th class="col col-8"><span

In [8]:
rows = table.find_all('tr', {'class': 'row'})[2:] # skip first 2 rows
rows

[<tr class="row row-2 even"><td class="col col-0 first">ABEVI228</td><td class="col col-1">ABEV        ON</td><td class="col col-2">set/19</td><td class="col col-3">22,57</td><td class="col col-4">-</td><td class="col col-5">-</td><td class="col col-6">-</td><td class="col col-7">-</td><td class="col col-8">-</td><td class="col col-9">-</td><td class="col col-10">0,75</td><td class="col col-11">-</td><td class="col col-12 last">-</td></tr>,
 <tr class="row row-3 odd"><td class="col col-0 first">ABEVH195</td><td class="col col-1">ABEV        ON</td><td class="col col-2">ago/19</td><td class="col col-3">19,23</td><td class="col col-4">-</td><td class="col col-5">-</td><td class="col col-6">-</td><td class="col col-7">-</td><td class="col col-8">-</td><td class="col col-9">-</td><td class="col col-10">0,30</td><td class="col col-11">-</td><td class="col col-12 last">-</td></tr>,
 <tr class="row row-4 even"><td class="col col-0 first">ABEVF160</td><td class="col col-1">ABEV        ON</td><

In [10]:
for i in rows[:10]:
    print(i.get_text())

ABEVI228ABEV        ONset/1922,57------0,75--
ABEVH195ABEV        ONago/1919,23------0,30--
ABEVF160ABEV        ONjun/1915,73-----0,05---
ABEVG165ABEV        ONjul/1916,57-----1,14---
ABEVF197ABEV        ONjun/1919,73------0,01--
ABEVG225ABEV        ONjul/1922,57------0,40--
ABEVF20ABEV        ONjun/1920,23------0,03--
ABEVF19ABEV        ONjun/1919,23------0,02--
ABEVF16ABEV        ONjun/1916,23-----0,85---
ABEVH182ABEV        ONago/1918,23-----0,100,70--


In [25]:
import re


# regexp = r"^([A-Z]{5}\d+)([A-Z]{4})\s+[O|P|N]{2,4}([a-z]{3}/\d{2})(\d+.\d+)-*(\d+.\d+)"
regexp = r"([A-Z]+\d+)([A-Z]{4}).*([a-z]{3}\/\d{2})(\d{1,2}\,\d{2})[\d]+\,[\d]{2}[\d]+\,[\d]{2}[\d]+\,[\d]{2}([\d]+\,[\d]{2})"

for i in rows[:1]:
    x = re.search(regexp, i.get_text())
    print(x.groups())

# ^([A-Z]{5}\d+)([A-Z]{4})\s+[O|P|N]{2,4}([a-z]{3}/\d{2})(\d+.\d+)-*(\d+.\d+)

AttributeError: 'NoneType' object has no attribute 'groups'

In [26]:
quotes = []
for i in rows:
    try:
        x = re.search(regexp, i.get_text())
        quotes.append(x.groups())
    except AttributeError:
        print(i.get_text())

ABEVI228ABEV        ONset/1922,57------0,75--
ABEVH195ABEV        ONago/1919,23------0,30--
ABEVF160ABEV        ONjun/1915,73-----0,05---
ABEVG165ABEV        ONjul/1916,57-----1,14---
ABEVF197ABEV        ONjun/1919,73------0,01--
ABEVG225ABEV        ONjul/1922,57------0,40--
ABEVF20ABEV        ONjun/1920,23------0,03--
ABEVF19ABEV        ONjun/1919,23------0,02--
ABEVF16ABEV        ONjun/1916,23-----0,85---
ABEVH182ABEV        ONago/1918,23-----0,100,70--
ABEVG195ABEV        ONjul/1919,57-----0,010,20--
ABEVG20ABEV        ONjul/1920,07------0,10--
ABEVG51ABEV        ONjul/1921,57------0,05--
ABEVJ182ABEV        ONout/1918,23-----0,01---
ABEVH48ABEV        ONago/1918,73------0,84--
ABEVH49ABEV        ONago/1919,73-----0,01---
ABEVJ217ABEVE       ONout/1921,48------0,39--
ABEVF184ABEVE       ONjun/1918,48------0,10--
ABEVF164ABEVE       ONjun/1916,48-----0,25---
ABEVI183ABEVE       ONset/1918,32-----0,08---
ABEVG183ABEVE       ONjul/1918,32-----0,020,52--
ARZZJ600ARZZ        ONout/1959,8

In [27]:
quotes

[('ABEVG16', 'ABEV', 'jul/19', '16,07', '1,77'),
 ('ABEVG250', 'ABEV', 'jul/19', '24,57', '0,03'),
 ('ABEVF180', 'ABEV', 'jun/19', '17,73', '0,23'),
 ('ABEVF185', 'ABEV', 'jun/19', '18,23', '0,06'),
 ('ABEVG180', 'ABEV', 'jul/19', '18,07', '0,41'),
 ('ABEVF187', 'ABEV', 'jun/19', '18,73', '0,01'),
 ('ABEVF17', 'ABEV', 'jun/19', '17,23', '0,50'),
 ('ABEVF46', 'ABEV', 'jun/19', '16,73', '0,91'),
 ('ABEVG185', 'ABEV', 'jul/19', '18,57', '0,22'),
 ('ABEVG170', 'ABEV', 'jul/19', '17,07', '0,92'),
 ('ABEVG47', 'ABEV', 'jul/19', '17,57', '0,62'),
 ('ABEVG19', 'ABEV', 'jul/19', '19,07', '0,12'),
 ('ABEVF172', 'ABEV', 'jun/19', '16,98', '0,70'),
 ('ABEVF177', 'ABEV', 'jun/19', '17,48', '0,36'),
 ('ABEVF179', 'ABEV', 'jun/19', '17,98', '0,10'),
 ('ABEVF199', 'ABEV', 'jun/19', '19,98', '0,02'),
 ('ABEVG178', 'ABEV', 'jul/19', '17,82', '0,49'),
 ('AZULF360', 'AZUL', 'jun/19', '36,00', '6,50'),
 ('BBASF505', 'BBAS', 'jun/19', '49,49', '2,41'),
 ('BBASF514', 'BBAS', 'jun/19', '50,49', '2,60'),
 ('BB

In [30]:
frame[frame["stock"]=="PETR"].sort_values(by=["exercise date", "series"])
frame = frame.filter(["stock", "exercise date", "series", "strike price"])
frame["call"] = frame["series"].str.get(4) < "L"
frame

,stock,exercise date,series,strike price,call
0,PETR,2019-09-16,PETRU197,18.94,False
1,SUZB,2019-06-17,SUZBF400,39.56,True
2,PETR,2019-08-19,PETRH102,9.48,True
3,RADL,2019-06-17,RADLR70,70.71,False
4,SBSP,2019-06-17,SBSPF45,44.01,True
5,BBDC,2020-01-20,BBDCM300,24.36,False
6,VALE,2020-04-20,VALED630,63.00,True
7,SUZB,2019-07-15,SUZBG323,32.31,True
8,SANB,2020-12-21,SANBX487,48.75,False
9,PETR,2020-01-20,PETRA307,30.65,True


In [41]:
frame["option price"] = 0.0
for q in quotes:
    frame.loc[frame["series"]==q[0], "option price"] = float(q[4].replace(",", "."))

frame

,stock,exercise date,series,strike price,call,option price
0,PETR,2019-09-16,PETRU197,18.94,False,0.00
1,SUZB,2019-06-17,SUZBF400,39.56,True,0.01
2,PETR,2019-08-19,PETRH102,9.48,True,0.00
3,RADL,2019-06-17,RADLR70,70.71,False,0.00
4,SBSP,2019-06-17,SBSPF45,44.01,True,0.00
5,BBDC,2020-01-20,BBDCM300,24.36,False,0.00
6,VALE,2020-04-20,VALED630,63.00,True,0.00
7,SUZB,2019-07-15,SUZBG323,32.31,True,0.00
8,SANB,2020-12-21,SANBX487,48.75,False,0.00
9,PETR,2020-01-20,PETRA307,30.65,True,0.00


In [37]:
q = quotes[0]
q[0]

'ABEVG16'

In [39]:
frame[frame["series"]==q[0]]

,stock,exercise date,series,strike price,call,option price
1536,ABEV,2019-07-15,ABEVG16,16.07,True,0.0


In [ ]:
options_frame = pd.DataFrame(columns=range(0,2), index = [0])

In [4]:
frame['implied volatility'] = frame.apply(lambda x: implied_volatility(x['value_1'],
                                                                       x['call'],
                                                                       29.85,
                                                                       x['strike price'],
                                                                       x['exercise date'],
                                                                       0.065), axis=1)

KeyError: ('value_1', 'occurred at index 0')